In [41]:
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np


In [42]:
ca = pd.read_csv(r'C:\Users\User\Documents\GitHub\Intensiv_3\Dataset\data\test.xlsx', encoding = 'latin2',  on_bad_lines='skip')

In [43]:
ca.head

<bound method NDFrame.head of                                                 PK  \
0   ?9LěŇČsbgŮŽ|Čl!őůUSh9i°br:"y_dlŔóDżý|-N...   
1                                       >~ţtśQú~ĄÔ=   
2                      ĆÇŐ%śŃź č\TŚC+Ů[ŕJŞďëĘcŞŹ   
3                                 aPJLWRiş( í-9E[   
4                                                ž$   
5   ŠďkžŽ{V:z'Ťh=ŕĂh¤Őh%â˝ítĎ-ÄĂłĽ(řM+]DŤ...   
6                                                      
7                                                jěe&   
8   ]Wľ( ö{˝űĂ˝gĐűŻ
ĺZRËGJZÚúGeŐ`r"FżůßZ...   
9   ý¤Ű.B¤Ů3Ç`÷xÎż2]ěÉ%ěćÚŕóŹ
ˇDŰ[!3ľŮĚĹĂ...   
10                                                  y   
11                                 ëŢd.ČĆáă×ßËtăgJZ   
12  g˝­CdśËůźü!2.¤óú/Âdć`%ă
Qů˙RĘnŹüë˙v{Ĺ...   
13                                       8D\tR'ŘnÔ   
14                                               6eÄ   
15                              šHąGŻDÝ­ŹV*Ó   
1